In [ ]:
import numpy as np
from wfdb import readsignal

In [ ]:
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999, physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l')  # .dat, .mat... 
print(sig)
print(fields)
#gains=np.array([1,2,3])
#sig=np.subtract(sig, gains)
#print(sig)

In [ ]:
# Testing multi-segment variable layout format
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31')

In [ ]:
np.sctypes

In [ ]:
a=np.array(range(0,12,4))
a=np.hstack((a,a))
print(a)

listofnps=[a,a]
print(listofnps)


listofnps.append("hello")
print(listofnps)



In [ ]:
numbs=['1','2','3']
num=[float(i) for i in numbs]
print(num)